In [1]:
from geopy.geocoders import Nominatim, GoogleV3
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import json
import pandas as pd
import numpy as np
import os
import time

#set the proxy so the geopy library can use it to visit external web sites
proxy_addr = '135.245.48.34:8000'
os.environ['http_proxy'] = proxy_addr
os.environ['https_proxy'] = proxy_addr

In [2]:
def get_geodata_single_metro_station(url):
    
    lat =''
    log=''
    station_page = requests.get(url, timeout=20)
    soup = BeautifulSoup(station_page.text, 'html.parser')
    #head_text= soup.head
    #print ("head_text =", head_text)
    geodata = soup.select('span.geo')
    #print ('geodata[0] = {}'.format(geodata[0]))
    if len(geodata)>0:
        item = geodata[0].get_text(strip=True)
        lat = item.split(';')[0].strip()
        log = item.split(';')[1].strip()
    print ('latitude: {}\nlogitude: {}'.format(lat, log))
    return lat, log

In [3]:
get_geodata_single_metro_station('https://en.wikipedia.org/wiki/Yishan_Road_station')

latitude: 31.186422
logitude: 121.427615


('31.186422', '121.427615')

In [4]:
def get_shanghai_metro_stations_information(read_from_csv=False):
    """
    It will first fetch the borugh and neighborhood table from wikipedia. Then go to each     link of the neighborhoods in the table. Finally, it will create a dataframe cotaining     Borough, Neighborhood and Population. It has a functionality to store the data in csv     format, and it is possible to read this data from csv to reduce time consuming            operations later.
    """
    csv = open('metro_shanghai.csv', 'a')
    #csv.write(";Shanghai metro station information: District,Station,Latitude,Longitude\n")
    print ("haha")
    if not read_from_csv:
        WIKI_LINK = "https://en.wikipedia.org/wiki/List_of_Shanghai_Metro_stations"
        ROOT_WIKI_LINK = "https://en.wikipedia.org"
        page = requests.get(WIKI_LINK, timeout = 10)
        soup = BeautifulSoup(page.text, 'html.parser')
        station_list = []
        soup_items = len(soup.select("table.wikitable tr"))
        print ('Number of items in the soup of the  pages = {}'.format(soup_items))
        test_row = 0
        count = 0
        district =''
        for table_row in soup.select("table.wikitable tr"):
            cells = table_row.findAll('td')
            #print ("table_row = soup.select(table.wikitable tr) = {}".format(table_row))
            #print ("cells = table_row.findAll(td) = {}".format(cells))
            print ("length of cells = {}\n\n".format(len(cells)))
            #test_row +=1
            #if test_row >15:
            #    print ("station_list contains {}".format(station_list))
            #    return
            if len(cells) > 5:
                station_found = False
                try:
                    for i in range(8):
                        print ("cells[i] = {}".format(cells[i]))
                        text = cells[i].text.strip().replace('\xa0',' ')
                        print ("text = cell[i].text.strip() ={}\n".format(text))
                        if len(text) > 3 and not station_found: 
                            if not ('/' in text): 
                                station = cells[i].text.strip().replace('\xa0', ' ')
                                station_found = True
                                station_link = cells[i].findAll('a')[0]['href']
                                lat, log = get_geodata_single_metro_station(ROOT_WIKI_LINK + station_link)
                                continue
                        if station_found:
                            link=cells[i].findAll('a')
                            if len(link)>0:
                                if ('District' in link[0]['href']):
                                    district = text.split('/')[0]
                                    print ("new district found = {}".format(district))
                                    break
                except:
                    pass
                if station_found:
                    count += 1
                    print ("Add station {} = {} - {}".format(count, district, station))
                    station_list.append([district, station, lat, log])
                    csv.write("{},{},{},{},{}\n".format(count,district,station,lat,log))
                    csv.flush()
                    time.sleep(30)

        #stations = list(dict.fromkeys(station_list))
        print ("station_list contains {}".format(station_list))
        print ("There are total {} stations in Shanghai Metro".format(len(station_list)))
        csv.close()
        return
        df = pd.DataFrame(station_list, columns=[
                          "Borough", "Neighborhood", "Population"])
        df.to_csv('population.csv')
    else:
        df = pd.read_csv('population.csv')
    df = df.sort_values(by=['Borough'])
    df = df.drop_duplicates(subset='Neighborhood', keep='last')
    return df


In [ ]:
get_shanghai_metro_stations_information()